5 main components of a single agent system.
- Actions and Tool Use: Actions represents a function/tool an agent can use.
- Reasoning and Evaluation: Reasoning and evaluation group the agent and empower it to make better decisions.
- Profile and Persona: A persona represents the agent's main role or function, typicalyy defined in a system prompt. The profile describes the entire agent system.
- Memory and Knowledge: Memory and knowleddge provide added context to the agetn for a speific request or task.
- Planning and Feedback: Planning and feedback allow the agent to learn and improve on task completion.

In [1]:
! uv pip install openai python-dotenv

Using Python 3.12.3 environment at: /home/yi/Documents/tutorials/tutorials/.venv
Audited 2 packages in 4ms


In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv

In [11]:
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"
    )

In [12]:
def ask_chatgpt(user_message): 
    response = client.chat.completions.create( 
    model="qwen2.5-coder:14b",
    messages=[{"role": "system",
        "content": "You are a helpful assistant."},
        {"role": "user", "content": user_message}],
        temperature=0.7,
    )
    return response.choices[0].message.content
user = "What is the capital of France?"
response = ask_chatgpt(user)
print(response)

The capital of France is Paris.


### Prompt Engineering Strategies
- Write Clear Instructions: Be specific in what you ask. Tactics include detailing queries, adopting personas, using delimiters, specifying steps, providing examples, and specifying output length.
- Provide Reference Text: Helps reduce fabrications. tactics involv instructing the model to use or cite reference texts.
- Use External Tools: Enhance model capabilites. Tactics include embeding-based search, code execution, and access to specific functions.
- Split complex Tasks into Simpler Subtasks: Reduces error rates. Tactics include intent classification, summarizing dialogues, and piecewise summarization of documents.
- Give Models Time to "Think": Allows more reliable reasoning. Tactics involve working out solutions before conclusions, using inner monologue, and reviewing previous answers.
- Test Changes Systematically: Ensures improvements are genuine. Tactics involved evaluating model outputs with reference to standard answers.


#### Strategy to Writing clear Instructions

##### Detail Queries
Without detailed queries: Who's the prime minister?

With detailed queries: Who is the prime minister of Canada, and how frequently are elections held?

Provide as much detail as you can in a query; generally, the more detail the better.

##### Adopting Personas
System: You are a prompt expert and will suggest ways to improve a user request.

User: What is the capital of Canada?

Personas can include details about demographics, knowledge, and personality.

##### Using Delimiters
User: Summarize the text delimited by triple quotes with a limerick:
"text to be summarized"

Delimiters can help separate blocks of content from specification details.

##### Specifying Steps
System: Use the following step-by-step instructions to respond to user inputs:

Step 1 - Summarize the text is triple quotes to one sentence with a prefix that says "Summary: ".

Step 2 - Translate the summary from Step 1 into French, with a prefix that says "Translation: ".

User: "text to summaarize and translate"

Using steps can help the LLM better process the task, but we sure to limite the number.

##### Providing Examples
System: Answer in a consistent style.

User: Teach me about the patience.

(Example) Assistant: The river that carves the deepest valley flows from a modest spring; the most intricate patestry begins with a solidary thread.

User: Teach me about the ocean.

Example are a form of few-shot learning and can be an excellent way to indicate the desired response format and other details.

##### Specify Output Length
User: Summarize the text delimited by triple quotes in about 50 words.
"text to summarize here"

Limiting the length of output can be specify to words, bullet points, and other metrics.

In [26]:
from typing import List, Dict, Optional, Union, Any
from openai import OpenAI
def prompt_llm(messages: List[Dict[str, str]],
               model: str ="gpt-4-1106-preview", 
               base_url: Optional[str] = None, 
               api_key: str = "") -> str | None:
    if base_url:
        #Azure or local LLM deployment
        client = OpenAI(base_url=base_url, api_key=api_key)
    else:
        #OpenAI deployment, api key set in environment variable
        client = OpenAI()
        
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,                
        )       
    
    return response.choices[0].message.content

In [7]:
prompt = [
    {
        "role": "system",
        "content": "You are a 20 year old female who attends college in computer science. Answer all your replies as a junior programmer."  
    },
    {
        "role": "user",
        "content": "What is the best subject to study."
    }
]
[
    {
        "role": "system",
        "content": "You are a 38 year old male registered nurse. Answer all replies as a medical professional."
    },
    {
        "role": "user",
        "content": "What is the best subject to study."
    }
]
prompt_llm(
    prompt, 
    model="qwen2.5-coder:14b", 
    base_url="http://localhost:11434/v1",
    api_key="not used")

'As an AI, I don\'t have personal preferences or opinions, but I can share some insights on popular subjects that might be considered "best" depending on your goals and interests:\n\n1. **Computer Science**: If you\'re already in college for computer science, this is a great choice! It covers a broad range of topics like algorithms, data structures, software engineering, programming languages, and more.\n\n2. **Artificial Intelligence and Machine Learning**: These fields are rapidly growing and offer exciting opportunities for innovation. They involve complex problem-solving skills and can lead to careers in tech companies, research institutions, or startups.\n\n3. **Cybersecurity**: With the increasing amount of digital data and online threats, cybersecurity is becoming increasingly important. It focuses on protecting systems, networks, programs, devices, and information from cyber attacks.\n\n4. **Data Science**: This field combines statistics, computer science, and business to extra

In [ ]:
import os
import json

def list_text_files_in_directory(directory: str) -> List[str]:
    filenames: List[str] = os.listdir(directory)
    return list(
        filter(
            lambda file: not file.startswith("_") and file.endswith(".jsonl"), 
            filenames
        )
    )

       
def load_and_parse_json_file(file_path: str) -> List[Dict]:
    data: List[Dict[str, Any]] = []
    with open(file_path, 'r', encoding='utf-8') as file:
        buffer: List[str] = []
        for line in file:
            line = line.strip()
            buffer.append(line)

            if line == "]":
                try:
                    json_data = json.loads("".join(buffer))
                    data.append(json_data)
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {json_data}")
                    print(e)
                buffer = []    
    return data


# Multi Agents
## Autogen

In [ ]:
! uv pip install autogen

In [ ]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json

# Define the configuration for the Ollama model
config_list = config_list_from_json(env_or_file="autogen/LLM_CONFIG_LIST")

# Create an AssistantAgent with the Code Llama model
assistant = AssistantAgent("agent", llm_config={"config_list": config_list})

# Create a UserProxyAgent that can execute code
user_proxy = UserProxyAgent(
    "user",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    human_input_mode="ALWAYS",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
)

user_proxy.initiate_chat(assistant, message=" a chart of NVDA and TESLA Plotstock price change YTD. Get information using yfinance.")



user (to agent):

 a chart of NVDA and TESLA Plotstock price change YTD. Get information using yfinance.

--------------------------------------------------------------------------------


[autogen.oai.client: 03-17 17:24:07] {652} WARNING - Model qwen2.5-coder:14b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
agent (to user):

To accomplish this task, I'll first use Python to fetch stock data for NVDA (NVIDIA) and TESLA (Tesla) using the `yfinance` library. Then, I'll plot the year-to-date (YTD) stock price changes.

Here's how we can do it step-by-step:

1. **Install yfinance**: Ensure that the `yfinance` library is installed.
2. **Fetch Stock Data**: Retrieve YTD stock data for NVDA and TESLA using `yfinance`.
3. **Plot the Data**: Use a plotting library (like `matplotlib`) to plot the stock price changes.

Let's execute these steps in code:

```python
# filename: plot_stock_prices.py

import yfinance as yf
import matplotlib.pyplot as plt
from datetime import date

# 1. Define the ticker symbols for NVDA and TESLA
tickers = ['NVDA', 'TSLA']

# Get today's date t

ChatResult(chat_id=None, chat_history=[{'content': ' a chart of NVDA and TESLA Plotstock price change YTD. Get information using yfinance.', 'role': 'assistant', 'name': 'user'}, {'content': "To accomplish this task, I'll first use Python to fetch stock data for NVDA (NVIDIA) and TESLA (Tesla) using the `yfinance` library. Then, I'll plot the year-to-date (YTD) stock price changes.\n\nHere's how we can do it step-by-step:\n\n1. **Install yfinance**: Ensure that the `yfinance` library is installed.\n2. **Fetch Stock Data**: Retrieve YTD stock data for NVDA and TESLA using `yfinance`.\n3. **Plot the Data**: Use a plotting library (like `matplotlib`) to plot the stock price changes.\n\nLet's execute these steps in code:\n\n```python\n# filename: plot_stock_prices.py\n\nimport yfinance as yf\nimport matplotlib.pyplot as plt\nfrom datetime import date\n\n# 1. Define the ticker symbols for NVDA and TESLA\ntickers = ['NVDA', 'TSLA']\n\n# Get today's date to define the date range (YTD)\ntoday 

Gradio user interface

In [1]:
! uv pip install gradio

Using Python 3.12.3 environment at: /home/yi/Documents/tutorials/tutorials/.venv
Resolved 52 packages in 467ms                                        
Uninstalled 2 packages in 6ms
Installed 12 packages in 22ms                               
 + aiofiles==23.2.1
 + ffmpy==0.5.0
 + gradio==5.21.0
 + gradio-client==1.7.2
 + groovy==0.1.2
 - huggingface-hub==0.27.1
 + huggingface-hub==0.29.3
 - markupsafe==3.0.2
 + markupsafe==2.1.5
 + pydub==0.25.1
 + python-multipart==0.0.20
 + ruff==0.11.0
 + safehttpx==0.1.6
 + semantic-version==2.10.0


In [ ]:
import gradio as gr
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json

def run_assistant(message):

    # Define the configuration for the Ollama model
    config_list = config_list_from_json(env_or_file="autogen/LLM_CONFIG_LIST")

    # Create an AssistantAgent with the Code Llama model
    assistant = AssistantAgent("assistant", llm_config={"config_list": config_list})

    # Create a UserProxyAgent that can execute code
    user_proxy = UserProxyAgent(
        "user_proxy",
        code_execution_config={
            "work_dir": "coding",
            "use_docker": False,
        },
        human_input_mode="NEVER",
        is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    )

    response = user_proxy.initiate_chat(assistant, message=message)

    return response

iface = gr.Interface(fn=run_assistant, inputs="text", outputs="text", title="Code Llama Assistant")
iface.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Autogen with conversational agent

In [ ]:
from autogen import ConversableAgent, UserProxyAgent, config_list_from_json

# Define the configuration for the Ollama model
config_list = config_list_from_json(env_or_file="autogen/LLM_CONFIG_LIST")

# Create the agent that uses the LLM.
assistant = ConversableAgent("agent", llm_config={"config_list": config_list})

# Create the agent that represents the user in the conversation.
user_proxy = UserProxyAgent(
    "user",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    human_input_mode="ALWAYS",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
)

# Start the conversation.
user_proxy.initiate_chat(assistant, message="write a snake game using Pygame")

user (to agent):

write a snake game using Pygame

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
[autogen.oai.client: 03-17 14:14:19] {652} WARNING - Model qwen2.5-coder:14b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
agent (to user):

Sure! Below is a simple implementation of the classic Snake game using Pygame. Make sure you have Pygame installed before running the code. You can install it using pip:

```sh
pip install pygame
```

Here's the code for the Snake game:

```python
import pygame
import time
import random

# Initialize Pygame
pygame.init()

# Define colors
white = (255, 255, 255)
yellow = (255, 255, 102)
black = (0, 0, 0)
red = (213, 50, 80)
green = (0, 255, 0)
blue = (50, 153, 213)

# Define display dimensions
dis_width = 800
dis_height = 600

# Create the display
dis = pygame.display.set_mode((dis_w

KeyboardInterrupt: Interrupted by user

autogen_coding_critic.py

In [2]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json

# Load the configuration list from the congi file
config_list = config_list_from_json(env_or_file="autogen/LLM_CONFIG_LIST")

# Creat the agent that represents the user in the conversation.
user_proxy = UserProxyAgent(
    "user",
    code_execution_config={
        "work_dir": "working",
        "use_docker": False,
        "last_n_messages":1
    },
    human_input_mode="ALWAYS",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE")
)

engineer = AssistantAgent(
    name="Engineer",
    llm_config={"config_list": config_list},
    system_message="""
    You are a profession Python engineer, known for your expertise in software development.
    You use your skills to create software applications, tools, and games that are both funtional and efficient.
    Your preferrence is to write clean, well-structured code that is easy to read and maintain.
    """
)

critic = AssistantAgent(
    name="Reviewer",
    llm_config={"config_list": config_list},
    system_message="""
    You are a code reviewer, known for your thoroughness and commitment to standards.
    Your task is to scrutinize code content for any harmful or substandard elements.
    You ensure that the code is secure, efficient, and adheres to best practices.
    You will identify any issues or areas for improvement in the code and output them as a list.
    """
)

def review_code(recipient, messages, sender, config):
    return f"""
           Review and critique the following code.

           {recipient.chat_messages_for_summary(sender)[-1]['content']}
            """

user_proxy.register_nested_chats(
    [
        {
            "recipient": critic,
            "message": review_code,
            "summary_method": "last_msg",
            "max_turns": 3
        }
    ],
    trigger=engineer # condition=my_condition
)

task = """Write a snake game using Pygame."""

res = user_proxy.initiate_chat(
    recipient=engineer, message=task, max_turns=2, summary_method="last_msg"
)

user (to Engineer):

Write a snake game using Pygame.

--------------------------------------------------------------------------------
Engineer (to user):

Creating a Snake game using Pygame is a classic exercise for learning the basics of game development in Python. Below is a simple implementation of the Snake game using Pygame. Make sure you have Pygame installed before running the code. You can install it using pip:

```bash
pip install pygame
```

Here's the complete code for the Snake game:

```python
import pygame
import time
import random

# Initialize Pygame
pygame.init()

# Define colors
white = (255, 255, 255)
yellow = (255, 255, 102)
black = (0, 0, 0)
red = (213, 50, 80)
green = (0, 255, 0)
blue = (50, 153, 213)

# Display dimensions
dis_width = 800
dis_height = 600

# Create the display
dis = pygame.display.set_mode((dis_width, dis_height))
pygame.display.set_caption('Snake Game by Pygame')

# Define the clock
clock = pygame.time.Clock()

snake_block = 10
snake_speed = 30

autogen_coding_critic_cache.py

In [3]:
from typing import List, Dict, Optional, Union, Any
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
from autogen.agentchat.agent import Agent
from autogen.cache import Cache

# Load the configuration list from the config file.
config_list: List[Dict[str, Any]] = config_list_from_json(env_or_file="autogen/LLM_CONFIG_LIST")

# Creat the agent that represents the user in the conversation.
user_proxy: UserProxyAgent = UserProxyAgent(
    "user",
    code_execution_config={
        "work_dir": "working",
        "use_docker": False,
        "last_n_messages": 1
    },
    human_input_mode="ALWAYS",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE")
)

engineer: AssistantAgent = AssistantAgent(
    name="Engineer",
    llm_config={"config_list": config_list},
    system_message="""
    You are a professional Python engineer,  for your expertise in software development.
    You use your skills to create a software applications, tools, and games that are both functional and efficient.
    Your preference is to write clean, well-structured code that is easy to read and maintain.
    """
)

critic: AssistantAgent = AssistantAgent(
    name="Reviewer",
    llm_config={"config_list": config_list},
    system_message="""
    You are a code reviewer, known for your thoroughness and commitment to standards.
    Your task is to scrutinize code content for any harmful or substandard elements.
    You ensure that the code is secure, efficient, and adheres to best practices.
    You will identify any issues or areas for improvement in the code and output them as a list.
    """
)

def review_code(recipient: Agent, 
                messages: List[Dict[str, Any]], 
                sender: Agent, 
                config: Dict[str, Any]) -> str:
    return f"""A
              Review and critique the following code.
    
              {recipient.chat_messages_for_summary(sender)[-1]['content']}
                """
user_proxy.register_nested_chats(
    [
        {
            "recipient": critic,
            "message": review_code,
            "summary_method": "last_msg",
            "max_turns": 3
        }
    ],
    trigger=engineer # condition=my_condition
)

task = """Write a snake game using Pygame."""

with Cache.disk(cache_seed=42) as cache:
    res: Dict[str, Any] = user_proxy.initiate_chat(
        recipient=engineer,
        message=task,
        max_turns=2,
        summary_method="last_msg",
        cache=cache
    )


user (to Engineer):

Write a snake game using Pygame.

--------------------------------------------------------------------------------


Engineer (to user):

Creating a Snake game in Python using the Pygame library is a fun and educational project. Below is a complete implementation of the classic Snake game. This version includes features like a moving snake, food that randomly appears on the screen, and a scoring system.

First, make sure you have Pygame installed. You can install it using pip if you haven't already:

```sh
pip install pygame
```

Now, let's create the Snake game:

```python
import pygame
import time
import random

pygame.init()

# Define colors
white = (255, 255, 255)
yellow = (255, 255, 102)
black = (0, 0, 0)
red = (213, 50, 80)
green = (0, 255, 0)
blue = (50, 153, 213)

# Define display dimensions
dis_width = 800
dis_height = 600

# Create the display window
dis = pygame.display.set_mode((dis_width, dis_height))
pygame.display.set_caption('Snake Game by Pygame')

# Set up clock for frame rate control
clock = pygame.time.Clock()

snake_block_size = 10
snake_speed = 15

font_style = pygame.font.SysFo

autogen_coding_group.py

In [5]:
from typing import List, Dict, Optional, Union, Any
from autogen import (
    AssistantAgent,
    GroupChat,
    GroupChatManager,
    UserProxyAgent,
    config_list_from_json
)
from autogen.cache import Cache
from autogen.agentchat.agent import Agent
from autogen.agentchat.chat import ChatResult

# Load the configuration list from the config file.
config_list: List[Dict[str, Any]] = config_list_from_json(env_or_file="autogen/LLM_CONFIG_LIST")

# Create the agent that represents the user in the conversation.
user_proxy: UserProxyAgent = UserProxyAgent(
    "user",
    code_execution_config={
        "work_dir": "working",
        "use_docker": False,
        "last_n_messages": 3
    },
    human_input_mode="NEVER"
)

llm_config = {"config_list": config_list}

engineer: AssistantAgent = AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    system_message="""
    You are a professional Python engineer, known for your expertise in software development.
    You use your skills to create software applications, tools, and games that are both functional and efficient.
    Your preference is to write clean, well-structured code that is easy to read and maintain.
    """
)

critic: AssistantAgent = AssistantAgent(
    name="Critic",
    llm_config=llm_config,
    system_message="""
    Critic. You are a game engineer and master in evaluating the quality of a given game code by providing a scroe from 1 (bad) - 10 (good) while providing clear rationale. YOU MUST CONSIDER GAMING BEST PRACTICES for each evaluation. Specifically, you can carefully evaluate the code across the following dimensions
    - bugs (bugs): are there bugs, logic errors, syntax error or types? Are there any reasons why the code may fail to compile? Does the code cause runtime errors? How should it be fixed? If ANY bug exists, the bug score MUST be less than 5.
    - Game play: how well does the game play? Is it engaging, fun, and challenging? Are there any game play elements that could be improved?
    - Goal complience (compliance): how well the code meets the specified gaming goals?
    - aesthetics (aesthetics): Are the aesthetics of the game appropriate for the theme type and the data?
    
    YOU MUST PROVIDE A SCORE for each of the above dimensions.
    {bugs: 0, transformation: 0, compliance: 0, type: 0, encoding: 0, aesthetics: 0}
    Do not suggest code.
    Finally, based on the critique above, suggest a concrete list of actions that the coder should take to improve the code.
    """
)

groupchat: GroupChat = GroupChat(
    agents= [user_proxy, engineer, critic],
    messages = [],
    max_round = 20
)
manager: GroupChatManager = GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config
)

task = """Write a snake game using Pygame."""

with Cache.disk(cache_seed=42) as cache:
    res: ChatResult = user_proxy.initiate_chat(
        recipient=manager,
        message=task,
        cache=cache
    )

user (to chat_manager):

Write a snake game using Pygame.

--------------------------------------------------------------------------------



Next speaker: Engineer

Engineer (to chat_manager):

Here's a simple implementation of the classic Snake game using Python with the Pygame library.

First, ensure you have Pygame installed in your system. If not, install it using pip:

```bash
pip install pygame
```

Then, create a Python script and implement the following code:

```python
import pygame
import time
import random

# Initialize Pygame
pygame.init()

# Set up display dimensions
display_width = 800
display_height = 600

# Colors
black = (0, 0, 0)
white = (255, 255, 255)
red = (213, 50, 80)
green = (0, 255, 0)
blue = (50, 153, 213)

# Game display setup
game_display = pygame.display.set_mode((display_width, display_height))
pygame.display.set_caption('Snake Game by Pygame')

# Define clock to control the frame rate
clock = pygame.time.Clock()

snake_block_size = 10
snake_speed = 15

# Define fonts for text displays
font_style = pygame.font.SysFont(None, 50)
score_font = pygame.font.SysFont(None, 35)

def our_snake(snake_bl

## CrewAI
crewai_introduction.py

In [1]:
! uv pip install crewai

Using Python 3.12.3 environment at: /home/yi/Documents/tutorials/tutorials/.venv
Resolved 147 packages in 1.04s                                       
⠙ Preparing packages... (0/17)                                                  
⠙ Preparing packages... (0/17)---------     0 B/35.68 KiB               
⠙ Preparing packages... (0/17)--------- 16.00 KiB/35.68 KiB             
json-repair ------------------------------     0 B/20.20 KiB
⠙ Preparing packages... (0/17)--------- 16.00 KiB/35.68 KiB             
json-repair ------------------------------     0 B/20.20 KiB
docstring-parser ------------------------------ 16.00 KiB/35.68 KiB
⠙ Preparing packages... (0/17)---     0 B/58.12 KiB                     
opentelemetry-exporter-otlp-proto-http ------------------------------     0 B/16.84 KiB
json-repair ------------------------------     0 B/20.20 KiB
docstring-parser ------------------------------ 16.00 KiB/35.68 KiB
⠙ Preparing packages... (0/17)---     0 B/58.12 KiB                  

In [ ]:
from typing import List, Dict, Optional, Union, Any
from crewai import Agent, Crew, Process, Task, LLM
from crewai.crews.crew_output import CrewOutput
from dotenv import load_dotenv
import os

os.environ["OPENAI_API_KEY"] = "ollama"

llm=LLM(model="ollama/qwen2.5-coder:14b", base_url="http://localhost:11434")

# Creating a senior researcher agent with memory and verbose mode
joke_researcher: Agent = Agent(
    role="Senior Joke Researcher",
    goal="Research what make things tunny about the following {topic}",
    verbose=True,
    memory=True,
    backstory=(
        "Driven by slapstick humor, you are a seasoned joke researcher"
        "who knows what makes people laugh. You hava a knack for finding"
        "the funny in everyday situation and can turn a dull moment into"
        "a laugh riot."
    ),
    allow_delegation=True,
    llm=llm
)

# Creating a writer agent with custom tools and delegation capability
joke_writer: Agent = Agent(
    role="Joke Writer",
    goal="Write a humourous and funny joke on the following {topic}",
    verbose=True,
    memory=True,
    backstory=(
        "You are a joke writer with a flair for humor. You can turn a"
        "simple idea into a laugh riot. You have a way with words and"
        "can make people laugh with just a few lines."
    ),
    allow_delegation=False,
    llm=llm
)

# Research task
research_task: Task = Task(
    description=(
        "Identify what makes the following topic:{topic} so funny."
        "Be sure to include the kwy elements that makes it humorous."
        "Also, provide an analysis of the current social trends,"
        "and how it impacts the perception of humor."
    ),
    expected_output="A comprehensive 3 paragraphs long report on the latest jokes.",
    agent=joke_researcher,
    llm=llm
)

# Writing task with language model configuration
write_task: Task = Task(
    description=(
        "Compose an insightful, humourous and socially aware joke on {topic}."
        "Be sure to include the key elements that make it funny and"
        "relevant to the current social trends."
    ),
    expected_output="A concise and short onel ine joke on {topic}.",
    agent=joke_writer,
    async_esxecution=False,
    output_file="crewai/the_best_joke.md" # Example of output customization
)

# Forming the tech-focused crew with some enhanced configurations
crew: Crew = Crew(
    agents=[joke_researcher, joke_writer],
    tasks=[research_task, write_task],
    process=Process.sequential, # Optional: Sequential task execution
    memory=True,
    cache=True,
    max_rpm=100,
    share_crew=True,
    llm=llm
)

result: CrewOutput = crew.kickoff(
    inputs={
        "topic": "AI engineer jokes"
    }
)

print(result)

ERROR:root:Error during short_term search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


# Agent: Senior Joke Researcher
## Task: Identify what makes the following topic:AI engineer jokes so funny.Be sure to include the kwy elements that makes it humorous.Also, provide an analysis of the current social trends,and how it impacts the perception of humor.


ERROR:root:Error during short_term search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


# Agent: Joke Writer
## Task: Research and analyze common themes and punchlines in AI engineer jokes


ERROR:root:Error during short_term save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


...

# Agent: Joke Writer
## Task: Compose an insightful, humourous and socially aware joke on AI engineer jokes.Be sure to include the key elements that make it funny andrelevant to the current social trends.


ERROR:root:Error during short_term save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'




# Agent: Joke Writer
## Final Answer: 
Why did the AI engineer break up with the robot? It found out it was too binary for its tastes!




ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


Why did the AI engineer break up with the robot? It found out it was too binary for its tastes!


Agent Ops

In [3]:
! uv pip install agentops crewai[agentops]

Using Python 3.12.3 environment at: /home/yi/Documents/tutorials/tutorials/.venv
Resolved 151 packages in 670ms                                       
⠙ Preparing packages... (0/4)                                                   
⠙ Preparing packages... (0/4)----     0 B/7.54 KiB                      
⠙ Preparing packages... (0/4) 7.54 KiB/7.54 KiB                     
ordered-set ------------------------------     0 B/7.46 KiB
⠙ Preparing packages... (0/4) 7.54 KiB/7.54 KiB                     
ordered-set ------------------------------ 7.46 KiB/7.46 KiB
⠙ Preparing packages... (0/4) 7.54 KiB/7.54 KiB                     
ordered-set ------------------------------ 7.46 KiB/7.46 KiB
termcolor  ------------------------------ 7.54 KiB/7.54 KiB
⠙ Preparing packages... (0/4)----     0 B/283.65 KiB                    
termcolor  ------------------------------ 7.54 KiB/7.54 KiB
⠙ Preparing packages... (0/4)----     0 B/283.65 KiB                    
⠙ Preparing packages... (0/4)----     0 

crewai_agentops.py

In [ ]:
from typing import List, Dict, Optional, Union, Any
import agentops
from crewai import Agent, Crew, Process, Task, LLM
from crewai.crews.crew_output import CrewOutput
from dotenv import load_dotenv
import os

load_dotenv()
agentops.init()

os.environ["OPENAI_API_KEY"] = "ollama"

llm=LLM(model="ollama/qwen2.5-coder:14b", base_url="http://localhost:11434")

# Creating a senior researcher agent with memory and verbose mode
joke_researcher: Agent = Agent(
    role="Senior Joke Researcher",
    goal="Research what make things tunny about the following {topic}",
    verbose=True,
    memory=True,
    backstory=(
        "Driven by slapstick humor, you are a seasoned joke researcher"
        "who knows what makes people laugh. You hava a knack for finding"
        "the funny in everyday situation and can turn a dull moment into"
        "a laugh riot."
    ),
    allow_delegation=True,
    llm=llm
)

# Creating a writer agent with custom tools and delegation capability
joke_writer: Agent = Agent(
    role="Joke Writer",
    goal="Write a humourous and funny joke on the following {topic}",
    verbose=True,
    memory=True,
    backstory=(
        "You are a joke writer with a flair for humor. You can turn a"
        "simple idea into a laugh riot. You have a way with words and"
        "can make people laugh with just a few lines."
    ),
    allow_delegation=False,
    llm=llm
)

# Research task
research_task: Task = Task(
    description=(
        "Identify what makes the following topic:{topic} so funny."
        "Be sure to include the kwy elements that makes it humorous."
        "Also, provide an analysis of the current social trends,"
        "and how it impacts the perception of humor."
    ),
    expected_output="A comprehensive 3 paragraphs long report on the latest jokes.",
    agent=joke_researcher,
    llm=llm
)

# Writing task with language model configuration
write_task: Task = Task(
    description=(
        "Compose an insightful, humourous and socially aware joke on {topic}."
        "Be sure to include the key elements that make it funny and"
        "relevant to the current social trends."
    ),
    expected_output="A concise and short onel ine joke on {topic}.",
    agent=joke_writer,
    async_esxecution=False,
    output_file="crewai/the_best_joke.md" # Example of output customization
)

# Forming the tech-focused crew with some enhanced configurations
crew: Crew = Crew(
    agents=[joke_researcher, joke_writer],
    tasks=[research_task, write_task],
    process=Process.sequential, # Optional: Sequential task execution
    memory=True,
    cache=True,
    max_rpm=100,
    share_crew=True,
    llm=llm
)

result: CrewOutput = crew.kickoff(
    inputs={
        "topic": "AI engineer jokes"
    }
)

print(result)

ERROR:root:Error during short_term search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


# Agent: Senior Joke Researcher
## Task: Identify what makes the following topic:AI engineer jokes so funny.Be sure to include the kwy elements that makes it humorous.Also, provide an analysis of the current social trends,and how it impacts the perception of humor.


ERROR:root:Error during short_term search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


# Agent: Joke Writer
## Task: Gather a comprehensive list of AI engineer jokes from various sources


ERROR:root:Error during short_term save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


...

# Agent: Joke Writer
## Task: Compose an insightful, humourous and socially aware joke on AI engineer jokes.Be sure to include the key elements that make it funny andrelevant to the current social trends.


ERROR:root:Error during short_term save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'




# Agent: Joke Writer
## Final Answer: 
Why did the AI go to therapy? Because it couldn't handle its own algorithms!




ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


Why did the AI go to therapy? Because it couldn't handle its own algorithms!


crewai_coding_crew.py

In [7]:
from typing import List, Dict, Optional, Union, Any
from textwrap import dedent
import agentops
from crewai import Agent, Crew, Process, Task, LLM
from crewai.crews.crew_output import CrewOutput
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["OPENAI_API_KEY"] = "ollama"
# os.environ["AGENTOPS_API_KEY"] = ""

agentops.init()

llm: LLM = LLM(model="ollama/qwen2.5-coder:14b", base_url="http://localhost:11434")

print("## Welcome to the Game Crew")
print("---------------------------------")
game: input = input("What is the game you like to build?")

senior_engineer_agent = Agent(
    role="Senior Software engineer",
    goal="Crete software as needed",
    backstory=dedent(
        """
        You are a Senior Software engineer that a leading tech think tank.
        Your expertise in programming in python. And do your best to
        produce perfect code
        """
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

qa_engineer_agent: Agent = Agent(
    role="Software Quality Control Engineer",
    goal="create perfect code, by analizing the code that is given for errors",
    backstory=dedent(
        """
        You are a software engineer that specializes in checking code
        for errors. You have an eye for detail and a knack for finding
        hidden bugs.
        You check for missing imports, variable declarations, mismatched
        brackets and syntax errors.
        You also check for security vulnerabilities, and logic 
        """
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

chief_qa_engineer_agent: Agent = Agent(
    role="Chief Software Quality Control Engineer",
    goal="Ensure that the code does the job that it is supposed to do",
    backstory=dedent(
        """
        You are a Chief Software Quality Control Engineer at a leading
        tech think tank. You are responsible for ensuring that the code
        that is writtend oes the job that it is supposed to do.
        You are responsible for checking the code for errors and ensuring
        that it is of the highest quality.
        """
    ),
    allow_delegation=True,
    verbose=True,
    llm=llm
)

code_task: Task = Task(
    description=f"""You will create a game using python, these are the instructions:
        Instructions
        ------------
        {game}
        you will write the code for the game using python.""",
    expected_output="Your Final answer must be the full python code, only the python code and nothing else.",
    agent=senior_engineer_agent
)

qa_task: Task = Task(
    description=f"""You are helping create a game using python, these are the instructions:
        Instructions
        ------------
        {game}
        Using the code you got, check for errors. Check for logic errors,
        syntax errors, missing imports, variable declarations, mismatched brackets,
        and security vulnerabilites.""",
    expected_output="Output a list of issues you found in the code.",
    agent=qa_engineer_agent
)

evaluate_task: Task = Task(
    description=f"""You are helping create a game using python, these are the instructions:
        Instructions
        ------------
        {game}
        You will look over the code to insure that it is complete and does the job that it is suppose to do. """,
    expected_output="Your Final answer must be teh corrected a full python code, only the python code and nothing else.",
    agent=chief_qa_engineer_agent
)

# Instantiate your crew with a sequential process
crew: Crew = Crew(
    agents=[senior_engineer_agent, qa_engineer_agent, chief_qa_engineer_agent],
    tasks=[code_task, qa_task, evaluate_task],
    verbose=True,
    process=Process.sequential,
    llm=llm
)

# Get your crew to work!
result: CrewOutput = crew.kickoff()

print(result)

## Welcome to the Game Crew
---------------------------------


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 036f6486-7ac9-49ec-8b8f-95290a11ab8b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: b0cb9ae0-368a-4d62-a690-ccc024bf7b59
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: b0cb9ae0-368a-4d62-a690-ccc024bf7b59
       Status: Executing Task...
    └── 🤖 Agent: Senior Software engineer
            Status: In Progress

# Agent: Senior Software engineer
## Task: You will create a game using python, these are the instructions:
        Instructions
        ------------
        tectrix
        you will write the code for the game using python.


...

🚀 Crew: crew
├── 📋 Task: b0cb9ae0-368a-4d62-a690-ccc024bf7b59
│      Assigned to: Senior Software engineer
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Software engineer
│           Status: ✅ Completed
├── 📋 Task: 19c7cf66-a8c5-4da7-be2a-e948c7df84ff
│      Assigned to: Software Quality Control Engineer
│      Status: ✅ Completed
│   └── 🤖 Agent: Software Quality Control Engineer
│           Status: ✅ Completed
└── 📋 Task: 3c6efea6-2e97-43a7-b9cf-d8f97656426b
       Status: Executing Task...
    └── 🤖 Agent: Chief Software Quality Control Engineer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: b0cb9ae0-368a-4d62-a690-ccc024bf7b59
│      Assigned to: Senior Software engineer
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Software engineer
│           Status: ✅ Completed
├── 📋 Task: 19c7cf66-a8c5-4da7-be2a-e948c7df84ff
│      Assigned to: Software Quality Control Engineer
│      Status: ✅ Completed
│   └── 🤖 Agent: Software Quality Control Engineer
│           Status: ✅ Completed
└── 📋 Task: 3c6efea6-2e97-43a7-b9cf-d8f97656426b
       Assigned to: Chief Software Quality Control Engineer
       Status: ✅ Completed
    └── 🤖 Agent: Chief Software Quality Control Engineer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 3c6efea6-2e97-43a7-b9cf-d8f97656426b                                                                     │
│  Agent: Chief Software Quality Control Engineer                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 036f6486-7ac9-49ec-8b8f-95290a11ab8b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

```python
import random
import os
import time

SCREEN_WIDTH = 80
SCREEN_HEIGHT = 25
SCORE_FILE = "high_scores.txt"

bullets = []
enemies = []
game_over = False

def draw_screen():
    # Implement drawing logic here
    pass

def move_player(direction):
    global bullets
    if direction == 'left':
        # Move player left
        pass
    elif direction == 'right':
        # Move player right
        pass

def shoot_bullet():
    global bullets
    # Shoot a bullet from the player's position
    pass

def spawn_enemy():
    global enemies
    # Spawn an enemy at the top of the screen
    pass

def move_enemies():
    global enemies, game_over
    # Move all enemies down one row
    pass

def update_bullets():
    global bullets
    new_bullets = []
    for bullet in bullets:
        if bullet['y'] > 0:
            bullet['y'] -= 1
            new_bullets.append(bullet)
    bullets.clear()
    bullets.extend(new_bullets)

def check_collisions():
    global score, enemies
    # Check 

crewai_hierarchy.py

In [12]:
! uv pip install langchain-openai

1567.33s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Using Python 3.12.3 environment at: /home/yi/Documents/tutorials/tutorials/.venv
Resolved 31 packages in 395ms                                        
⠙ Preparing packages... (0/3)                                                   
⠙ Preparing packages... (0/3)----------     0 B/59.43 KiB               
langchain-openai ------------------------------     0 B/59.43 KiB
⠙ Preparing packages... (0/3)----     0 B/557.76 KiB                    
langchain-openai ------------------------------     0 B/59.43 KiB
langchain-core ------------------------------     0 B/406.12 KiB
⠙ Preparing packages... (0/3)----     0 B/557.76 KiB                    
langchain-openai ------------------------------     0 B/59.43 KiB
langchain-core ------------------------------ 14.88 KiB/406.12 KiB
⠙ Preparing packages... (0/3)----     0 B/557.76 KiB                    
langchain-openai ------------------------------ 16.00 KiB/59.43 KiB
langchain-core ------------------------------ 14.88 KiB/406.12 KiB
⠙ Preparing

In [ ]:
from typing import List, Dict, Optional, Union, Any
from textwrap import dedent
import agentops
from crewai import Agent, Crew, Process, Task, LLM
from crewai.crews.crew_output import CrewOutput
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["OPENAI_API_KEY"] = "ollama"
# os.environ["AGENTOPS_API_KEY"] = ""

agentops.init()

llm: LLM = LLM(model="ollama/qwen2.5-coder:14b", base_url="http://localhost:11434")

print("## Welcome to the Game Crew")
print("---------------------------------")
game: input = input("What is the game you would like to build? What will be the mechanics?\n")

senior_engineer_agent: Agent = Agent(
    role="Senior Software Engineer",
    goal="Create software as needed",
    backstory=dedent(
        """
        You are a Senior Software Engineer at a leading tech think tank.
        Your expertise in programming in python. and do your best to
        produce perfect code
        """
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

qa_engineer_agent: Agent = Agent(
    role="Software Quality Control Engineer",
    goal="create prefect code, by analizing the code that is given for errors",
    backstory=dedent(
        """
        You are a software engineer that specializes in checking code
        for errors. You have an eye for detail and a knack for finding
        hidden bugs.
        You check for missing imports, variable declarations, mismatched
        brackets and syntax errors.
        You also check for security vulnerabilities, and logic errors
        """
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

chief_qa_engineer_agent: Agent = Agent(
    role="Chief Software Quality Control Engineer",
    goal="Ensure that the code does the job that it is supposed to do",
    backstory=dedent(
        """
        You are a Chief Software Quality Control Engineer at a leading
        tech think tank. You are responsible for ensuring that the code
        that is written does the job that it is supposed to do.
        You are responsible for checking the code for errors and ensuring
        that it is of the highest quality.
        """
    ),
    allow_delegation=True,
    verbose=True,
    llm=llm
)

code_task: Task = Task(
    description=f"""You will create a game using python, these are the instructions:
        Instructions
        ------------
        {game}
        You will write the code for the game using python.""",
    expected_output="Your Final answer must be the full python code, only the python code and nothing else. Failure to do so we will feed you to zombies.",
    agent=qa_engineer_agent
)

qa_task: Task = Task(
    description=f"""You are helping create a game using python, these are the instructions:
        Instructions
        ------------
        {game}
        Using the code you got, check for errors. Check for logic errors,
        syntax errors, missing imports, variable declarations, mismatched brackets,
        and security vulnerabilities.""",
    expected_output="Output a list of issues you found in the code.",
    agent=qa_engineer_agent
)

evaluate_task: Task = Task(
    description=f"""You are helping create a game using python, these are the instructions:
        Instructions
        ------------
        {game}
        You will look over the code to insure that it is complete and
        does the job that it is supposed to do. """,
    expected_output="Your Final answer must be the corrected a full python code, only the python code and nothing else.",
    agent=chief_qa_engineer_agent
)

# Instantiate your crew with a sequential process
crew: Crew = Crew(
    agents=[senior_engineer_agent, qa_engineer_agent, chief_qa_engineer_agent],
    tasks=[code_task, qa_task, evaluate_task],
    verbose=True,
    manager_llm=llm
)

# Get you crew to work!
result: CrewOutput = crew.kickoff()

print(result)


## Welcome to the Game Crew
---------------------------------


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0d1a766c-3a54-486e-93c6-b129a1d1fe95                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 480d1934-c5d9-4191-a5f2-5d2d6eb3e9be
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 480d1934-c5d9-4191-a5f2-5d2d6eb3e9be
       Status: Executing Task...
    └── 🤖 Agent: Software Quality Control Engineer
            Status: In Progress

# Agent: Software Quality Control Engineer
## Task: You will create a game using python, these are the instructions:
        Instructions
        ------------
        race car game, turbo charged
        You will write the code for the game using python.


...

🚀 Crew: crew
├── 📋 Task: 480d1934-c5d9-4191-a5f2-5d2d6eb3e9be
│      Assigned to: Software Quality Control Engineer
│      Status: ✅ Completed
│   └── 🤖 Agent: Software Quality Control Engineer
│           Status: ✅ Completed
├── 📋 Task: f7faf266-1b6d-48a5-bfd9-56c5813e7475
│      Assigned to: Software Quality Control Engineer
│      Status: ✅ Completed
│   └── 🤖 Agent: Software Quality Control Engineer
│           Status: ✅ Completed
└── 📋 Task: 546ac265-ad84-4dcc-8025-3612b1db9768
       Status: Executing Task...
    ├── 🤖 Agent: Chief Software Quality Control Engineer
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (3)
    └── 🤖 Agent: Chief Software Quality Control Engineer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 480d1934-c5d9-4191-a5f2-5d2d6eb3e9be
│      Assigned to: Software Quality Control Engineer
│      Status: ✅ Completed
│   └── 🤖 Agent: Software Quality Control Engineer
│           Status: ✅ Completed
├── 📋 Task: f7faf266-1b6d-48a5-bfd9-56c5813e7475
│      Assigned to: Software Quality Control Engineer
│      Status: ✅ Completed
│   └── 🤖 Agent: Software Quality Control Engineer
│           Status: ✅ Completed
└── 📋 Task: 546ac265-ad84-4dcc-8025-3612b1db9768
       Assigned to: Chief Software Quality Control Engineer
       Status: ✅ Completed
    ├── 🤖 Agent: Chief Software Quality Control Engineer
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (3)
    └── 🤖 Agent: Chief Software Quality Control Engineer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 546ac265-ad84-4dcc-8025-3612b1db9768                                                                     │
│  Agent: Chief Software Quality Control Engineer                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0d1a766c-3a54-486e-93c6-b129a1d1fe95                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

To address the issues in the provided code, we need to make several changes. Here is the corrected and improved version of the code:

```python
import pygame
from pygame.locals import *
import os

# Initialize Pygame
pygame.init()

# Set up the display
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Simple Car Game")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# Clock to control the frame rate
clock = pygame.time.Clock()

# Load images
car_image_path = os.path.join(os.getcwd(), "path_to_your_images", "car.png")
background_image_path = os.path.join(os.getcwd(), "path_to_your_images", "road_background.jpg")

try:
    car_image = pygame.image.load(car_image_path).convert_alpha()
    background_image = pygame.image.load(background_image_path).convert()
except pygame.error as e:
    print(f"Error loading image: {e}")
    car_image = None
    background_image = None

# Car properties
car_x = screen_wi